In [1]:
import sys
import pandas as pd
import torch
import pyro
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

sys.path.insert(0, '/home/djl34/lab_pd/kl/git/KL')

import raklette_updated

####################################################
# Create Dataset
####################################################
class TSVDataset(Dataset):
    def __init__(self, path, chunksize, nb_samples, header_all, features):
        self.path = path
        self.chunksize = chunksize
        self.len = nb_samples // self.chunksize
        self.header = header_all
        self.features = features
        
    def __getitem__(self, index):
        x = next(
            pd.read_csv(
                self.path,
                sep = "\t",
                skiprows=index * self.chunksize + 1,  #+1, since we skip the header
                chunksize=self.chunksize,
                names=self.header))
        
        x = x[self.features]
        x["e_module"] = x["e_module"] + 1
        x = torch.from_numpy(x.values)
        return x

    def __len__(self):
        return self.len

KL_data_dir = "/home/djl34/lab_pd/kl/data/"
scratch_dir = "/n/scratch3/users/d/djl34/"

/home/djl34/.conda/envs/kl/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

sfs = pd.read_csv(KL_data_dir + "neutral_SFS_5bins.tsv", sep = "\t")

bin_columns = []

for i in range(5):
    bin_columns.append(str(i) + "_bin")

neutral_sfs = torch.tensor(sfs[bin_columns].values)

mu_ref = torch.tensor(sfs["mu"].values)

df = pd.read_csv(scratch_dir + "kl_input/enhancer_module/22.tsv", sep = "\t")

nb_samples = len(df)
header = df.columns

dataset = TSVDataset(scratch_dir + "kl_input/enhancer_module/22.tsv", chunksize=100000, nb_samples = nb_samples, header_all = header, features = header)

loader = DataLoader(dataset, batch_size=1, num_workers=1, shuffle=False)

# df = df.sort_values("e_module")

# df["e_module"] = df["e_module"] + 1

# mu_vals = torch.tensor(df["mu_index"].values)
# gene_ids = torch.tensor(df["e_module"].values)
# gene_ids = gene_ids.type(torch.LongTensor)
# sample_sfs = torch.tensor(df["Freq_bin"].values)

# covariates = torch.tensor([1]*len(mu_vals))
# covariates = covariates.unsqueeze(0).transpose(0,1).type(torch.FloatTensor)

#define variables
#define neut_sfs_full, mu_vals, gene_ids, covariates
# n_covs = covariates.shape[-1]          # number of covariates included
# n_genes = len(torch.unique(gene_ids)) + 1  # number of genes

n_genes = len(df["e_module"].unique()) + 1
n_covs = 0

# n_covs = 1
# n_genes = 3
n_bins = len(neutral_sfs[1]) - 1

#define model and guide
KL = raklette_updated.raklette(neutral_sfs, n_bins, mu_ref, n_covs, n_genes)
model = KL.model
guide = pyro.infer.autoguide.AutoNormal(model)

#run inference
pyro.clear_param_store()
# run SVI
adam = pyro.optim.Adam({"lr":0.005})
elbo = pyro.infer.Trace_ELBO(num_particles=1, vectorize_particles=True)
svi = pyro.infer.SVI(model, guide, adam, elbo)
losses = []

num_epochs = 1

In [3]:
df.head()

,mu_index,Freq_bin,e_module
0,75,0,-1.0
1,86,1,-1.0
2,887,4,-1.0
3,253,0,-1.0
4,240,0,-1.0


In [4]:
nb_samples

77201997

In [5]:
loader

In [6]:
for batch_idx, data in enumerate(loader):
    break

In [7]:
data.shape

torch.Size([1, 100000, 3])

In [8]:
data[:, :, 0].shape

torch.Size([1, 100000])

In [10]:
data[:,:,0].reshape(-1).shape

torch.Size([100000])

In [30]:
data[:, :, 1]

tensor([[0., 1., 4.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [33]:
data[:, :, 2]

tensor([[0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [6]:
for epoch in range(num_epochs):
    # Take a gradient step for each mini-batch in the dataset
    for batch_idx, data in enumerate(loader):
        loss = svi.step(data[:,:,0].reshape(-1), data[:,:,2].reshape(-1), None, data[:,:,1].reshape(-1))
#         if y is not None:
#             y = y.type_as(x)
#         loss = svi.step(x, y)
        losses.append(loss)
    break

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/djl34/.conda/envs/kl/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/djl34/.conda/envs/kl/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/djl34/.conda/envs/kl/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_815/1404713150.py", line 34, in __getitem__
    x = torch.from_numpy(x.data.values)
  File "/home/djl34/.conda/envs/kl/lib/python3.10/site-packages/pandas/core/generic.py", line 5907, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'DataFrame' object has no attribute 'data'


In [4]:
x

NameError: name 'x' is not defined